# Notebook to run gPhoton
Run gPhoton to create GALEX light curves etc.
Setup your source parameter and setting in the cell below.

In [1]:
from astropy.io import ascii
from astropy import units as uu

src_str = """label	ra	dec
SDSS _142118.18+523547.1 215.3258439968967	52.596545351520945
PB_5130 334.61900740472345 -0.0033813554025151345
GALEX_J221409.8+005245 333.5413675869932 0.8794347888579439"""
run_src_idx = 2

tt_srcs = ascii.read(src_str)
outdir = "./resources/gPhoton_out/"
outfile_fin = outdir+"gPhoton_"+str(tt_srcs["label"][run_src_idx])+'_fin.npy'
outfile_app = outdir+"gPhoton_"+str(tt_srcs["label"][run_src_idx])+'_app.npy'
ron = (6*uu.arcsec).to(uu.deg).value        # Radius of signal annulus
roff1 = (10*uu.arcsec).to(uu.deg).value      # Inner radius of background ring
roff2 = (15*uu.arcsec).to(uu.deg).value      # Outer radius of background ring
#load_past_results = False

In [2]:
%matplotlib widget
import gPhoton
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii
from astropy.table import Table
import os
from vasca.utils import mag2flux, flux2mag, tgalex_to_astrotime
import vasca.visualization as vvis
from vasca.region import Region

### Do aperture photometry for each visit with gPhoton

In [ ]:
%%time
pos_ra = tt_srcs["ra"][run_src_idx]
pos_dec = tt_srcs["dec"][run_src_idx]

#Run or load gFind
if os.path.isfile(outfile_fin):
    dd_gfind = np.load(outfile_fin,allow_pickle='TRUE').item()
    t_bins = list(zip(dd_gfind["NUV"]["t0"],dd_gfind["NUV"]["t1"]))
    print("Number of time bins:",len(t_bins))
else:
    print(f"Running query with gFind for:\n",tt_srcs[run_src_idx])
    dd_gfind = gPhoton.gFind(band='NUV',skypos=[pos_ra,pos_dec]) #,maxgap=100.,minexp=100.
    t_bins = list(zip(dd_gfind["NUV"]["t0"],dd_gfind["NUV"]["t1"]))
    np.save(outfile_fin,dd_gfind)

#Run or load gApperture
if os.path.isfile(outfile_app):
    dd_gph = np.load(outfile_app,allow_pickle='TRUE').item()
else:
    print("Running lightcurve with gAperture..")
    dd_gaperture = gPhoton.gAperture(band='NUV',skypos=[pos_ra,pos_dec],radius=ron, annulus=[roff1,roff2],tranges=t_bins)
    dd_gph = {"gAperture":dd_gaperture,"gFind":dd_gfind}
    np.save(outfile_app,dd_gph)
    print("..done")


Running query with gFind for:
 label         ra               dec        
----- ----------------- ------------------
WD900 215.3258439968967 52.596545351520945
NUV: 61701.0s (raw) in 48 exposures.
    [ 775893225.995, 775893315.995 ], 90.000 seconds
    [ 775899147.995, 775899237.995 ], 90.000 seconds
    [ 777313477.995, 777315163.995 ], 1686.000 seconds
    [ 777354890.995, 777355470.995 ], 580.000 seconds
    [ 903243480.995, 903245052.995 ], 1572.000 seconds
    [ 903480942.995, 903481678.995 ], 736.000 seconds
    [ 903586710.995, 903588176.995 ], 1466.000 seconds
    [ 903906287.995, 903907726.995 ], 1439.000 seconds
    [ 904917809.995, 904919506.995 ], 1697.000 seconds
    [ 906195657.995, 906197231.995 ], 1574.000 seconds
    [ 906326883.995, 906327407.995 ], 524.000 seconds
    [ 906409485.995, 906410247.995 ], 762.000 seconds
    [ 933993847.995, 933995405.995 ], 1558.000 seconds
    [ 934183202.995, 934184686.995 ], 1484.000 seconds
    [ 934354830.995, 934356250.995 ], 142

In [ ]:
#print(dd_gph["gAperture"].keys())
#print("\n", dd_gph["gFind"]["NUV"].keys())
#print(dd_gph["gAperture"]['photons'])
keep_keys = ('t_mean', 'exptime','flux_bgsub', 'flux_bgsub_err','mag_bgsub','mag_bgsub_err_2',"flags","mag_mcatbgsub","mag_mcatbgsub_err_2", "mag")
dd_gap = {x: dd_gph["gAperture"][x] for x in keep_keys if x in dd_gph["gAperture"]}
dd_gap["s2n"]=dd_gap['flux_bgsub']/dd_gap['flux_bgsub_err']

# Units of flux_bgsub are in erg sec^-1 cm^-2 Å^-1. . Get also Jy flux from AB magnitude
dd_gap['flux'],dd_gap['flux_err'] = mag2flux(dd_gap["mag_mcatbgsub"],dd_gap["mag_mcatbgsub_err_2"])
dd_gap['t_mean']= tgalex_to_astrotime(dd_gap['t_mean'],"mjd")
# Units of time are in "GALEX Time" = "UNIX Time" - 315964800, change to MJD

tt_gap = Table(data= dd_gap)
sel_gap = tt_gap["s2n"]>3.5
display(tt_gap[sel_gap])

### Display lightcurve

In [ ]:
region_name = "WD" #"MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
rg = Region()
rg.load_from_fits(region_fname)
rg_src_id= 142

fig_lc_src = plt.figure("Light curve",figsize=(8, 4), clear=True)
vvis.plot_light_curve(rg, rg_src_ids = rg_src_id,fig=fig_lc_src, flux_var="flux_auto") #
plt.show()


#fig = plt.figure(figsize=(8, 6), constrained_layout=True)
plt.errorbar(tt_gap[sel_gap]['t_mean'],tt_gap[sel_gap]['flux'],yerr=tt_gap[sel_gap]['flux_err'],fmt=".")
plt.yscale('log')

#fig2 = plt.figure(figsize=(8, 6), constrained_layout=True)
#ax = plt.gca()
#ax.invert_yaxis()
#plt.errorbar(tt_gap[sel_gap]['t_mean'],tt_gap[sel_gap]['mag_bgsub'],yerr=tt_gap[sel_gap]['mag_bgsub_err_2'],fmt="bo")

In [ ]:
# Calculate apperture flux correctin from http://www.galex.caltech.edu/researcher/techdoc-ch5.html
f1 = mag2flux(19)
f2 = mag2flux(19-0.20)
mag = flux2mag(f2)
print(f1,f2,f2/f1, mag)
